[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uwsampl/tutorial/blob/master/notebook/02_TVM_Tutorial_Relay.ipynb)

Please follow the introduction of the TVM tutorial before running this. 
The below code assumes you have already setup TVM, and merely loads it from your Google Drive.

In [3]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')

In [4]:
%%bash
[[ ! -e /tools/google-cloud-sdk ]] && exit
echo "Installing Dependencies ..."
sudo apt-get update
sudo apt-get install -y -q llvm-6.0 libglfw3-dev libtinfo-dev libffi-dev zlib1g-dev clinfo
cd "/content/gdrive/My Drive"
if [[ ! -e tvm ]]; then
    echo "Cloning TVM ..."
    git clone --recursive https://github.com/dmlc/tvm
fi
echo "Configuring Build ..."
cd tvm
mkdir -p build
cp cmake/config.cmake build
# sed -i -e 's/USE_OPONGL OFF/USE_OPONGL ON/g' build/config.cmake
sed -i -e 's/USE_CUDA OFF/USE_CUDA ON/g' build/config.cmake
sed -i -e 's/USE_CUDNN OFF/USE_CUDNN ON/g' build/config.cmake
sed -i -e 's/USE_LLVM OFF/USE_LLVM ON/g' build/config.cmake
sed -i -e 's/USE_VTA_TSIM OFF/USE_VTA_TSIM ON/g' build/config.cmake
cat build/config.cmake
echo "Running CMake ..."
cd build
cmake ..
echo "Building TVM ..."
make -j4
cd ..
echo "Installing Python libraries ..."
cd "/content/gdrive/My Drive/tvm/"
cd python; python setup.py install; cd ..
cd topi/python; python setup.py install

# Relay: an extensible deep learning IR

TODO fill in text

## Hetergenous Execution

In [5]:
import tvm
from tvm import relay
import tvm.relay.testing
from tvm.relay.expr_functor import ExprMutator

class ScheduleConv2d(ExprMutator):
    def __init__(self, device):
        self.device = device
        super().__init__()

    def visit_call(self, expr):
        visit = super().visit_call(expr)
        if expr.op == tvm.relay.op.get("nn.conv2d"):
            return relay.annotation.on_device(visit, self.device)
        else:
            return visit

def schedule_conv2d_on_gpu(expr):
    sched = ScheduleConv2d(tvm.gpu(0))
    return sched.visit(expr)

# TODO(@jroesch): use pass manager
resnet, params = relay.testing.resnet.get_workload()
print(resnet)
resnet = schedule_conv2d_on_gpu(resnet)
print(resnet)
resnet = relay.ir_pass.rewrite_annotated_ops(resnet, 0)
print(resnet)

v0.0.1
fn (%data: Tensor[(1, 3, 224, 224), float32], %bn_data_gamma: Tensor[(3,), float32], %bn_data_beta: Tensor[(3,), float32], %bn_data_moving_mean: Tensor[(3,), float32], %bn_data_moving_var: Tensor[(3,), float32], %conv0_weight: Tensor[(64, 3, 7, 7), float32], %bn0_gamma: Tensor[(64,), float32], %bn0_beta: Tensor[(64,), float32], %bn0_moving_mean: Tensor[(64,), float32], %bn0_moving_var: Tensor[(64,), float32], %stage1_unit1_bn1_gamma: Tensor[(64,), float32], %stage1_unit1_bn1_beta: Tensor[(64,), float32], %stage1_unit1_bn1_moving_mean: Tensor[(64,), float32], %stage1_unit1_bn1_moving_var: Tensor[(64,), float32], %stage1_unit1_conv1_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_bn2_gamma: Tensor[(64,), float32], %stage1_unit1_bn2_beta: Tensor[(64,), float32], %stage1_unit1_bn2_moving_mean: Tensor[(64,), float32], %stage1_unit1_bn2_moving_var: Tensor[(64,), float32], %stage1_unit1_conv2_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_sc_weight: Tensor[(64, 64, 1, 1)